In [15]:
# validate the verion (3.5 version is more compatible with minio)
# jupyter-pyspark image: quay.io/jupyter/pyspark-notebook:spark-3.5.1
import pyspark 
print("PySpark:", pyspark.__version__) 
print("Spark:", spark.version) 
print("Hadoop:", spark.sparkContext._jsc.hadoopConfiguration().get("hadoop.version"))
print(spark._jsc.hadoopConfiguration().get("fs.s3a.endpoint"))

PySpark: 3.5.1
Spark: 3.5.1
Hadoop: None
http://minio.airflow:9000


In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = (
    SparkSession.builder
    .appName("spark-minio")
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio.airflow:9000")
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .getOrCreate()
)

In [12]:
data = [
    ("James", "Smith", "36636", "M", 3000),
    ("Michael", "Rose", "40288", "M", 4000),
    ("Robert", "Williams", "42114", "M", 4000),
    ("Maria", "Jones", "39192", "F", 4000),
    ("Jen", "Brown", "39193", "F", -1)
]

schema = StructType(
    [
        StructField("first_name", StringType(), True),
        StructField("last_name", StringType(), True),
        StructField("id", StringType(), True),
        StructField("gender", StringType(), True),
        StructField("salary", IntegerType(), True),
    ]
)

df = spark.createDataFrame(data, schema)

In [13]:
df.show()

+----------+---------+-----+------+------+
|first_name|last_name|   id|gender|salary|
+----------+---------+-----+------+------+
|     James|    Smith|36636|     M|  3000|
|   Michael|     Rose|40288|     M|  4000|
|    Robert| Williams|42114|     M|  4000|
|     Maria|    Jones|39192|     F|  4000|
|       Jen|    Brown|39193|     F|    -1|
+----------+---------+-----+------+------+



In [14]:
# write minio
df.write.mode("overwrite").parquet("s3a://datalake/bronze/jupyter-pyspark-iceberg-0-1-0-test-2")